In [1]:
DATA_NAME = 'spaceNet-full' 
TRANSFORM = 'wavelet-horizVert'
CHANNEL = 'blue'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_blue_horizVert_wavelet_full_spaceNet.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9
2,17099.99,0.09,0.14,-0.04,0.04,0.02,-0.00,-0.00
3,0.09,4440.04,-0.11,-0.01,0.01,-0.00,0.00,-0.00
4,0.14,-0.11,1375.03,-0.01,0.00,0.01,0.00,0.00
5,-0.04,-0.01,-0.01,383.33,-0.00,-0.01,-0.00,-0.00
6,0.04,0.01,0.00,-0.00,87.51,0.00,0.00,0.00
7,0.02,-0.00,0.01,-0.01,0.00,18.37,-0.00,0.00
8,-0.00,0.00,0.00,-0.00,0.00,-0.00,3.83,0.00
9,-0.00,-0.00,0.00,-0.00,0.00,0.00,0.00,0.72


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9
2,1.00000,0.00001,0.00003,-0.00001,0.00003,0.00004,-0.00002,-0.00002
3,0.00001,1.00000,-0.00004,-0.00001,0.00002,-0.00001,0.00002,-0.00006
4,0.00003,-0.00004,1.00000,-0.00001,0.00001,0.00009,0.00000,0.00000
5,-0.00001,-0.00001,-0.00001,1.00000,-0.00003,-0.00006,-0.00001,-0.00001
6,0.00003,0.00002,0.00001,-0.00003,1.00000,0.00002,0.00002,0.00001
7,0.00004,-0.00001,0.00009,-0.00006,0.00002,1.00000,-0.00002,0.00010
8,-0.00002,0.00002,0.00000,-0.00001,0.00002,-0.00002,1.00000,0.00002
9,-0.00002,-0.00006,0.00000,-0.00001,0.00001,0.00010,0.00002,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.2954848013994065

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[3.66099798e+07 2.41817408e+06 2.25199500e+05 1.67934759e+04
 8.21908403e+02 3.23598973e+01 1.07633225e+00 0.00000000e+00]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7
0,0.999144,-0.039625,-0.011404,-0.003162,-0.000719,-0.000150,-0.000032,-0.000006
1,0.038858,0.997493,-0.057291,-0.014306,-0.003236,-0.000680,-0.000141,-0.000027
2,0.013382,0.055788,0.996398,-0.061085,-0.012701,-0.002644,-0.000554,-0.000104
3,0.004470,0.017309,0.059232,0.996153,-0.060830,-0.011985,-0.002493,-0.000470
4,0.001265,0.004879,0.015800,0.058828,0.995313,-0.073482,-0.014472,-0.002720
5,0.000349,0.001350,0.004348,0.015670,0.070551,0.991271,-0.108502,-0.019131
6,0.000107,0.000411,0.001327,0.004768,0.020889,0.101700,0.976630,-0.188121
7,0.000042,0.000160,0.000515,0.001852,0.008103,0.038586,0.184946,0.981956


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.0008557314715023079,
 0.0025070494268304433,
 0.003602178986480409,
 0.003846620499541764,
 0.0046870580734396095,
 0.008729378458440906,
 0.023369854500111198,
 0.01804439250962475]